In [2]:
import numpy as np
import sys
sys.path.append("../")

import information_decomposition as pid
from intersection_information import intersection_information

In [3]:
# Parameter definitions for the examples
sigma = 0.5 # prior probability of s=1
rho = 0.1 # bit-flip probability for S->R channel
gamma = 0.1 # bit-flip probability for S->C channel where this exists

In [4]:
# Joint PDF table definitions for the examples

# example where SI is only due to signal correlations, as R<-S->C
P1 = np.zeros((2,2,2))
P1[0,0,0] = sigma*(1-rho)*(1-gamma)
P1[0,0,1] = sigma*(1-rho)*(gamma)
P1[0,1,0] = sigma*rho*(1-gamma)
P1[0,1,1] = sigma*rho*gamma
P1[1,0,0] = (1-sigma)*rho*gamma
P1[1,0,1] = (1-sigma)*rho*(1-gamma)
P1[1,1,0] = (1-sigma)*(1-rho)*(gamma)
P1[1,1,1] = (1-sigma)*(1-rho)*(1-gamma)

# example that should have nonzero intersection, as S->R==C
P2 = np.zeros((2,2,2))
P2[0,0,0] = sigma*(1-rho)
P2[0,0,1] = 0
P2[0,1,0] = 0
P2[0,1,1] = sigma*rho
P2[1,0,0] = (1-sigma)*rho
P2[1,0,1] = 0
P2[1,1,0] = 0
P2[1,1,1] = (1-sigma)*(1-rho)

In [58]:
# Compute and print intersection information, together with "non-normalised" and "signal correlations-only" SI values
print(intersection_information(P1))
print(intersection_information(P2))

(0.0, 0.31992237501539894, 0.31992237501539894)
(0.21108203139531984, 0.53100440641071878, 0.31992237501539894)


## Introducing IDTxl

In [7]:
import idtxl
from idtxl.estimators_pid import synergy_tartu

In [10]:
# my decomposition for the P2 problem above, as a sanity check
[SI, CI, UI_Y, UI_Z] = pid.decomposition(P2)
print(SI, CI, UI_Y, UI_Z)

0.531004406411 0.0 0.0 0.0


In [59]:
# convert definition of P2 for tartu
P = dict()
for x in [0,1]:
    for y in [0,1]:
        for z in [0,1]:
            P[(x,y,z)] = P2[x,y,z]
print(P)

{(0, 1, 1): 0.050000000000000003, (0, 0, 0): 0.45000000000000001, (1, 1, 0): 0.0, (1, 0, 0): 0.050000000000000003, (0, 1, 0): 0.0, (0, 0, 1): 0.0, (1, 1, 1): 0.45000000000000001, (1, 0, 1): 0.0}


In [60]:
[the_p, feas, kkt, wCI, wSI, wUI_Y, wUI_Z] = synergy_tartu.solve_PDF(pdf=P)
print(wSI, wCI, wUI_Y, wUI_Z)

0.5310044058278642 5.82854542408e-10 0.0 0.0


## Example: dice

In [55]:
aa=np.int(0);

lstep = 0.05
l=0
l_n = np.int(np.ceil(1/lstep)+1)

I_shar=np.zeros(l_n);
I_uny=np.zeros(l_n);
I_unz=np.zeros(l_n);
I_syn=np.zeros(l_n);

I_shar_z=np.zeros(l_n);
I_uny_z=np.zeros(l_n);
I_unz_z=np.zeros(l_n);
I_syn_z=np.zeros(l_n);

#Y and Z are the dice, X is the weighted sum of the two


# summed-dice dimensions
dimy=np.int(6);
dimz=np.int(6);
 
alpha=np.int(1); # relative ratio between the sum coefficients for summing the dice
 
dimx=np.int(np.ceil(dimy+dimz*alpha-1-alpha)+1);



#make X the new target
dimx_x=dimz;
dimy_x=dimy;
dimz_x=dimx;


while aa<=np.ceil(1/lstep): # loop over different correlations between the dice

    
    p_yz = np.zeros((dimy,dimz))
    
    # setting correlations between the dice
    for yy in range(0,dimy):
        for zz in range(0,dimz):
            p_yz[yy,zz]=l/36+(1-l)/6*np.float(zz==yy)
    
    p_yz=p_yz/p_yz.sum() # p(y,z), joint probability of the two dice alone

    p_x_yz=np.zeros((dimx,dimy,dimz)) # p(x|y,z)

    #build p(x|y,z) analytically
    for xx in range(0,dimx):
        for yy in range(0,dimy):
            for zz in range(1,dimz):
                if xx==np.ceil(zz+alpha*yy)-alpha:
                    p_x_yz[xx,yy,zz]=1
   
    p=np.zeros((dimx,dimy,dimz))

    #build p analytically
    for xx in range(1,dimx):
        for yy in range(1,dimy):
            for zz in range(1,dimz):
                p[xx,yy,zz]=p_x_yz[xx,yy,zz]*p_yz[yy,zz]

    p=p/p.sum() #final normalization

    # adapt definitions to those expected by tartu
    P = dict()
    P_z = dict()
    for x in range(0,dimx):
        for y in range(0,dimy):
            for z in range(0,dimz):
                P[(x,y,z)] = p[x,y,z]
                P_z[(z,y,x)] = p[x,y,z]

    %time [_, _, _, I_syn[aa],I_shar[aa],I_uny[aa],I_unz[aa]] = synergy_tartu.solve_PDF(pdf=P)
    #%time [_, _, _, I_syn_z[aa],I_shar_z[aa],I_uny_z[aa],I_unz_z[aa]] = synergy_tartu.solve_PDF(pdf=P_z)
    

    aa+=1
    l+=lstep
    

CPU times: user 6.22 s, sys: 12.5 s, total: 18.7 s
Wall time: 5.31 s
CPU times: user 6.85 s, sys: 14.3 s, total: 21.2 s
Wall time: 6.28 s
CPU times: user 6.51 s, sys: 12.8 s, total: 19.4 s
Wall time: 5.48 s
CPU times: user 6.28 s, sys: 12.6 s, total: 18.9 s
Wall time: 5.35 s
CPU times: user 6.67 s, sys: 12.7 s, total: 19.4 s
Wall time: 5.47 s
CPU times: user 6.6 s, sys: 13.6 s, total: 20.2 s
Wall time: 5.86 s
CPU times: user 6.62 s, sys: 13.3 s, total: 19.9 s
Wall time: 5.84 s
CPU times: user 6.81 s, sys: 15.4 s, total: 22.2 s
Wall time: 7.11 s
CPU times: user 7.19 s, sys: 15.3 s, total: 22.5 s
Wall time: 7.03 s
CPU times: user 7.63 s, sys: 16.1 s, total: 23.7 s
Wall time: 7.03 s
CPU times: user 7.33 s, sys: 15.6 s, total: 22.9 s
Wall time: 6.62 s
CPU times: user 7.21 s, sys: 14.5 s, total: 21.7 s
Wall time: 6.13 s
CPU times: user 6.66 s, sys: 13.4 s, total: 20.1 s
Wall time: 5.63 s
CPU times: user 6.58 s, sys: 13 s, total: 19.6 s
Wall time: 5.5 s
CPU times: user 7.41 s, sys: 15 s, tot

In [56]:
print(I_syn)

[ -1.23092603e-09   2.79428218e-01   4.93239927e-01   6.82199511e-01
   8.54656695e-01   1.01440445e+00   1.16354020e+00   1.30332792e+00
   1.43454914e+00   1.55767080e+00   1.67293303e+00   1.78039626e+00
   1.87996495e+00   1.97139549e+00   2.05429135e+00   2.12808528e+00
   2.19200584e+00   2.24502112e+00   2.28574575e+00   2.31228182e+00
   2.32192810e+00]


In [57]:
print(I_shar)

[ 2.32192809  2.17662626  2.05717878  1.94728441  1.84360975  1.74464646
  1.64956723  1.5578816   1.46929629  1.38364901  1.30087456  1.22098763
  1.14407537  1.0702971   0.99989011  0.9331823   0.87061356  0.8127703
  0.76044171  0.71471535  0.67715147]
